In [1]:
import numpy as np
import pandas as pd
pd.set_option('max_columns',None)
pd.set_option('max_rows',None)

In [2]:
import s3fs
import pyarrow as pa
import pyarrow.parquet as pq

def spark_read_parquet(s3_url: str, **args):
    fs = s3fs.S3FileSystem()
    # Leyendo base
    dataset = pq.ParquetDataset(s3_url, filesystem=fs)
    table = dataset.read()
    dataframe = table.to_pandas()

    del dataset, table

    return dataframe

fs = s3fs.S3FileSystem()

## Filtrando libranzas

In [3]:
## Leyendo base libranzas ciclo de vida

mes = '01'
anio = '2022'

str_date = pd.to_datetime(anio+'-'+mes, format = '%Y-%m')
anio = str(int(anio)-1) if mes == '01' else anio
mes = str((str_date - np.timedelta64(20, 'D')).month) if (str_date - np.timedelta64(20, 'D')).month > 9 else '0' + str((str_date - np.timedelta64(20, 'D')).month)


path = 's3://data-bpop-dev-sandbox/estandarizado/productos/libranzas/productos_libranzas_dwh_M'+anio+mes
path

's3://data-bpop-dev-sandbox/estandarizado/productos/libranzas/productos_libranzas_dwh_M202112'

In [4]:
# Leyendo base
dataset = pq.ParquetDataset(path, filesystem=fs)
table = dataset.read()
lib = table.to_pandas()
lib = lib.drop_duplicates()

del dataset, table

In [5]:
print(lib.shape)
lib.head()

(3261817, 68)


,sk_cliente,id_cliente,sk_rc_libranza,id_rc_libranza,sk_fe_radicacion,sk_fe_aprobacion,sk_fe_desembolso,sk_fe_finalizacion,sk_fe_estado_actual,sk_convenio_libranza,sk_persona,dk_persona,sk_oficina,sk_producto_servicio,sk_clasificacion_tradicional,cd_oficina,cd_originador,ds_originador,cd_segmento_lib,ds_segmento_lib,cd_subsegmento_lib,ds_subsegmento_lib,no_solicitud,fe_solicitud,vl_monto_solicitado,mp_reestructurado,no_obligacion,fe_desembolso,vl_monto_aprobado,vl_recapitalizacion,vl_monto_desembolsado,no_cuotas,fe_finalizacion,no_meses_per_gracia,vl_total_cuota,fe_exigibilidad_cuota_1,cd_estado_actual,ds_estado_actual,fe_estado_actual,vl_tasa,vl_tasa_efectiva_anual,cd_modalidad_pag_int,ds_modalidad_pag_int,cd_periodicidad_pag_int,ds_periodicidad_pag_int,cd_base_liquidacion,cd_tipo_libranza,ds_tipo_libranza,cd_tipo_credito,ds_tipo_credito,cd_tipo_colocacion,ds_tipo_colocacion,no_obligacion_novada,vl_monto_novado,mp_compra_ind_cartera,mp_afecta_desp_nomina,cd_linea_credito,fe_carga,fe_actua,no_particion,cd_sector,cd_subsector,cd_tipo_pagaduria,ds_sector,ds_subsector,ds_tipo_pagaduria,standard_id,id_cli
0,60834,102652295584838601,1023924,1000003*11439693*LB*36403010026444,None,None,2004-02-05,2006-04-05,1900-01-01,5063,662495,1000003*11439693,166,27,None,364,1,Banco Popular,1,Defensa,1,Defensa,36403010028969,2004-02-05,2100000.0,0,36403010026444,2004-02-05,2100000.0,0.0,1555542,24,2006-04-05,0,113137,2004-04-05,99,CANCELADO,1900-01-01,26.00,25.7097,V,Vencido,M,Mensual,360,1,Organica,0000000000000000000000000000000000000000000000...,Novacion,1,1-ORD HST 50 AÃO 364 DIA,0,382878.0,0,0,66,2019-04-12,2021-08-25,0,1,4,1,ORDEN NACIONAL,Mindefensa Ejercito Nacional Activos,NACIONAL CENTRALIZADA,1000003,11439693
1,60834,102652295584838601,280423,1000003*11439693*LB*9550301121187,None,None,2001-06-12,2004-07-05,1900-01-01,19724,662495,1000003*11439693,None,27,None,955,1,Banco Popular,1,Defensa,1,Defensa,9550301121187,1900-01-01,3100000.0,0,9550301121187,2001-06-12,2960000.0,0.0,0,36,2004-07-05,0,128101,2001-07-05,99,CANCELADO,1900-01-01,31.50,25.7097,V,Vencido,M,Mensual,360,1,Organica,0000000000000000000000000000000000000000000000...,Nueva,1,1-ORD HST 50 AÃO 364 DIA,0,0.0,0,0,66,2019-04-12,2021-12-05,0,1,4,1,ORDEN NACIONAL,Mindefensa Ejercito Nacional Activos,NACIONAL CENTRALIZADA,1000003,11439693
2,60951,102652297314584801,390474,1000003*23823293*LB*25003010013115,None,None,2002-10-03,2005-07-05,1900-01-01,5354,1250337,1000003*23823293,151,27,None,250,1,Banco Popular,2,Educativo,2,Educativo,25003010013632,2002-09-27,4000000.0,0,25003010013115,2002-10-03,4000000.0,0.0,2104473,32,2005-07-05,0,175856,2002-11-05,99,CANCELADO,1900-01-01,26.60,25.7097,V,Vencido,M,Mensual,360,1,Organica,0000000000000000000000000000000000000000000000...,Novacion,1,1-ORD HST 50 AÃO 364 DIA,0,1697710.0,0,0,66,2019-04-12,2021-08-25,0,3,7,3,ENTES TERRITORIALES,Secretaría de Educacion,REGIONAL,1000003,23823293
3,60951,102652297314584801,215366,1000003*23823293*LB*2500300206080,None,None,2000-12-27,2004-02-05,1900-01-01,5354,1250337,1000003*23823293,151,27,None,250,1,Banco Popular,2,Educativo,2,Educativo,2500300206080,2000-12-27,2940000.0,0,2500300206080,2000-12-27,2940000.0,0.0,0,36,2004-02-05,0,125614,2001-02-05,99,CANCELADO,1900-01-01,30.50,25.7097,V,Vencido,M,Mensual,360,1,Organica,0000000000000000000000000000000000000000000000...,Nueva,1,1-ORD HST 50 AÃO 364 DIA,0,0.0,0,0,66,2019-04-12,2021-08-25,0,3,7,3,ENTES TERRITORIALES,Secretaría de Educacion,REGIONAL,1000003,23823293
4,60978,102652303931898101,2197892,1000003*78746146*LB*58703090012023,None,None,2015-07-30,2022-09-05,2021-09-20,23893,2575095,1000003*78746146,245,27,None,564,1,Banco Popular,5,Policia,5,Policia,58703090016332,2015-07-24,54500000.0,0,58703090012023,2015-07-30,54500000.0,0.0,2180428,84,2022-09-05,0,943608,2015-09-05,116,CASTIGO SISTEMATIZADO,2021-09-20,10.68,11.2186,V,Vencido,M,Mensual,360,1,Organica,0000000000000000000000000000000000000000000000...,Novacion,9,9-PEN 1 SM 75-79 AÃO 364 DIA,

In [6]:
lib = lib[lib['id_cli'].notnull()]
lib['id_cli'] = lib['id_cli'].astype('int')
lib['id_cliente'] = lib['id_cliente'].astype('str')
lib['standard_id'] = lib['standard_id'].astype('int')

In [7]:
lib_orig = lib.copy()

In [8]:
lib['ds_estado_actual'] = lib['ds_estado_actual'].str.upper()
ids = lib.loc[~lib['ds_estado_actual'].isin(['CANCELADO', 'EN COBRO JURIDICO', 'CASTIGO MANUAL', 
                                            'CASTIGO SISTEMATIZADO', 'CANCELADO POR REESTRUCTURADA']),['id_cli', 'standard_id','id_cliente']].drop_duplicates()

ids_final = ids.rename(columns = {'id_cli':'id_numero_cliente'})
ids_final['fecha_camp'] = int(anio+mes)
ids_final.shape

(326181, 4)

In [9]:
ids_final.head(15)

,id_numero_cliente,standard_id,id_cliente,fecha_camp
22,20300762,1000003,102652296820136301,202112
30,4963836,1000003,102652294887775501,202112
37,8487399,1000003,102652312101686601,202112
65,6389801,1000003,103552305523787102,202112
68,1049637963,1000003,103552317976910502,202112
71,28090689,1000003,102652310893707501,202112
83,40369544,1000003,102652298945622101,202112
98,94153890,1000003,103552305155659402,202112
110,9082613,1000003,102652316127356401,202112
116,4313783,1000003,102652294821537501,202112


In [10]:
# # Data Demog

# path = 's3://data-bpop-dev-sandbox/estandarizado/clientes/identificacion/clientes_identificacion_mdm_D20210218'
# data_demog = spark_read_parquet(path)
# data_demog = data_demog[data_demog['ref_num'].str.isdigit()]
# print(data_demog.shape)
# data_demog.head(5)

In [11]:
# # Join cedula, typeid
# data_demog = data_demog.rename(columns={'cont_id':'id_cliente',
#                                        'ref_num':'id_numero_cliente'})

# data_demog['id_tp_cd'] = data_demog['id_tp_cd'].astype('str')

# data_demog = data_demog.drop_duplicates(subset=['id_tp_cd', 'id_numero_cliente'], keep='first') ## Problemas de duplicados

In [12]:
# data_demog['id_cliente'] = data_demog['id_cliente'].fillna('-99').astype(np.int64)

In [13]:
# ids_final = pd.merge(ids_final,data_demog[['id_cliente','id_numero_cliente','id_tp_cd']], on='id_cliente', how='left')
# ids_final.count()

In [14]:
ids_final.head()

,id_numero_cliente,standard_id,id_cliente,fecha_camp
22,20300762,1000003,102652296820136301,202112
30,4963836,1000003,102652294887775501,202112
37,8487399,1000003,102652312101686601,202112
65,6389801,1000003,103552305523787102,202112
68,1049637963,1000003,103552317976910502,202112


In [15]:
"s3://adl-refined-dev-popular/parquet/TC_adquisicion/base_ini"+anio+mes

's3://adl-refined-dev-popular/parquet/TC_adquisicion/base_ini202112'

In [16]:
## Guardando bases
ids_final.to_parquet("s3://adl-refined-dev-popular/parquet/TC_adquisicion/base_ini"+anio+mes,engine='pyarrow', index=False)

In [17]:
aaaaaaaaa

NameError: name 'aaaaaaaaa' is not defined

In [ ]:
"""
MODULE: processing_filtering
This script extracts activos information from the activos file
Steps:
1. Get activos file
2. Get the needed columns and rows from activos file including the needed historical period
3. Create additional columns
"""

def processing_filtering(input_paths_activos):
    """Pending docstring"""
    # 1. Get filter file
    ids_final = get_filtering_file(input_paths_activos)
    print('get_filtering_file')
    # 2. Get demog file
    data_demog = get_demog_file(path)
    print('get_demog_file')
    # 3. Join between 
    ids_final2 = pd.merge(ids_final,data_demog[['id_cliente','id_numero_cliente','id_tp_cd']], on='id_cliente', how='left')
    print('ids_final_df')
    print('filtering_df created successfully')

    return ids_final2


def get_filtering_file(input_path):
    """ Gets the input_path to the activos file, drops some not useful columns
     and outputs a DataFrame
    :param input_path to activos file location
    :return: DataFrame
    """
    
    mes = '01'  ### REVISAR!!!!
    anio = '2021'   ### REVISAR!!!!
    
    filtering_file_path = input_path
    filtering = spark_read_parquet(filtering_file_path)

    filtering.rename(columns=lambda x: x.lower(), inplace=True)
    
    # Filter and casting variables
    filtering = filtering[filtering['id_cliente'].notnull()]
    filtering['id_cliente'] = filtering['id_cliente'].astype('int')
    
    ids = filtering.loc[~filtering['ds_estado_actual'].isin(['Cancelada','Castigado']),'id_cliente'].unique()
    ids_final = pd.DataFrame({'id_cliente':ids})
    ids_final['fecha_camp'] = int(anio+mes)  ### REVISAR!!!!

    return ids_final


def get_demog_file(path):
    """ Gets the input_path to the activos file, drops some not useful columns
     and outputs a DataFrame
    :param input_path to activos file location
    :return: DataFrame
    """
    
    print(path)
    # read the new table
    data_demog = spark_read_parquet(path)
    data_demog = data_demog[data_demog['ref_num'].str.isdigit()]

    # Join cedula, typeid and casting variables
    data_demog = data_demog.rename(columns={'cont_id':'id_cliente',
                                       'ref_num':'id_numero_cliente'})
    data_demog['id_tp_cd'] = data_demog['id_tp_cd'].astype('str')
    
    # Drop duplicates and casting variables
    data_demog = data_demog.drop_duplicates(subset=['id_tp_cd', 'id_numero_cliente'], keep='first')
    data_demog['id_cliente'] = data_demog['id_cliente'].fillna('-99').astype(np.int64)

    return data_demog

In [ ]:
mes = '01'
anio = '2021'
input_paths_activos = 's3://data-bpop-dev-sandbox/estandarizado/productos/libranzas/productos_libranzas_dwh_M'+anio+mes
path = 's3://data-bpop-dev-sandbox/estandarizado/clientes/identificacion/clientes_identificacion_mdm_D20210218'

ids_final2 = processing_filtering(input_paths_activos)

print(ids_final2.shape)
ids_final2.head()

In [18]:
#############33

In [19]:
path = 's3://data-bpop-dev-sandbox/casos-uso/propension-tc/output/Propension-tc_preaprob_M20211215_20211215_204118.csv'

data1 = pd.read_csv(path, sep='|')
print(data1.shape)
data1.head()

(326827, 9)


,fuente,id_cliente,id_numero_cliente,score,decil,nivel_prob,num_meses_ult_camp,camp_anteriores,grupo_camp
0,CRM-PS,102652295880533801,13544804,0.023246,4,Alto,NaN,No Tocado,3
1,CRM-PS,102652295151150201,6880226,0.029480,5,Alto,NaN,No Tocado,3
2,CRM-PS,102652314595088801,41311613,0.014120,3,Medio,NaN,No Tocado,3
3,CRM-PS,102652308842687701,36695801,0.004341,1,Bajo,NaN,No Tocado,1
4,CRM-PS,103552297266851902,23173372,0.002392,1,Bajo,NaN,No Tocado,1


In [20]:
path = 's3://data-bpop-dev-sandbox/casos-uso/propension-tc/output/Propension-tc_preaprob_M20220112_20220112_205335.csv'

data = pd.read_csv(path, sep='|')
print(data.shape)
data.head()

(326301, 9)


,fuente,id_cliente,id_numero_cliente,score,decil,nivel_prob,num_meses_ult_camp,camp_anteriores,grupo_camp
0,CRM-PS,102652296820136301,20300762,0.004738,2,Medio Bajo,4.066667,Tocado,2
1,CRM-PS,102652294887775501,4963836,0.007460,2,Medio Bajo,NaN,No Tocado,1
2,CRM-PS,102652312101686601,8487399,0.031049,5,Alto,NaN,No Tocado,3
3,CRM-PS,103552305523787102,6389801,0.039600,5,Alto,NaN,No Tocado,3
4,CRM-PS,103552317976910502,1049637963,0.008879,3,Medio,NaN,No Tocado,3


In [21]:
import os

os.getcwd()

'/mnt/work/CU_adquisicionTC/Notebooks/Pipeline'

In [22]:
path = '/mnt/work/CU_adquisicionTC/Notebooks/Pipeline/Data_Dic2021.csv'
data.to_csv(path, sep='|')

In [23]:
ids = pd.Series([5027017,22527235,1121817079,856194,8263395,8281618,22689679])

In [24]:
ids_final.head()

,id_numero_cliente,standard_id,id_cliente,fecha_camp
22,20300762,1000003,102652296820136301,202112
30,4963836,1000003,102652294887775501,202112
37,8487399,1000003,102652312101686601,202112
65,6389801,1000003,103552305523787102,202112
68,1049637963,1000003,103552317976910502,202112


In [32]:
lib_orig.loc[lib_orig['id_cli'].isin(ids),['id_cli','ds_estado_actual']]

,id_cli,ds_estado_actual
27488,8281618,EN COBRO JURIDICO
27489,8281618,CANCELADO
27490,8281618,CANCELADO
27491,8281618,CANCELADO
27492,8281618,CANCELADO
27493,8281618,CANCELADO
27494,8281618,CANCELADO
27495,8281618,CANCELADO
27496,8281618,CANCELADO
27497,8281618,CANCELADO


In [29]:
len(ids[ids.isin(lib_orig['id_cli'])])/ids.shape[0]

0.8571428571428571

In [26]:
len(ids[ids.isin(data['id_numero_cliente'])])/ids.shape[0]

0.0

In [27]:
len(ids_final[ids_final['id_numero_cliente'].isin(data1['id_numero_cliente'])])/ids_final.shape[0]

0.9615949426851963

In [28]:
len(ids_final[ids_final['id_numero_cliente'].isin(data['id_numero_cliente'])])/ids_final.shape[0]

1.0